In [1]:
import sys
sys.path.append('/home/taindp/PycharmProjects/thesis')

In [2]:
%pwd

'/home/taindp/PycharmProjects/thesis/jupyter'

In [3]:
import pandas as pd
from tqdm import tqdm

from entity.pattern_ner import find_all_entity
from intent.intent_regconize import catch_intent
from entity.confirm_object import catch_point

In [4]:
_PATH_QUESTION = '/home/taindp/PycharmProjects/thesis/data/question_livestream_label.csv'

In [5]:
question = pd.read_csv(_PATH_QUESTION)
question.head(1)

,label,content
0,1,thầy cho em hỏi nếu mình đã trúng tuyển chương...


In [6]:
def get_nlu(mess):
    try:
        intent_catched, prob,mess_clean = catch_intent(mess)
        dict_entity_inform, confirm_obj = find_all_entity(intent_catched,mess_clean)
        dict_invest = {}
        dict_invest['intent'] = intent_catched
        dict_invest['probabily'] = prob
        dict_invest['entity'] = dict_entity_inform
        dict_invest['confirm_obj'] = confirm_obj
        return dict_invest
    except:
        return {}

In [7]:
list_invest = []
for item in tqdm(list(question['content'])):
    dict_invest = get_nlu(item)
    list_invest.append(dict_invest)

100%|██████████| 433/433 [00:05<00:00, 73.23it/s]


In [8]:
question['invest'] = list_invest

In [9]:
for idx,item in enumerate(list_invest):
    item['text'] = list(question['content'])[idx]
    item['labeled_intent'] = list(question['label'])[idx]

In [10]:
list_invest

[{'intent': 'type_edu',
  'probabily': 0.9992730021476746,
  'entity': {'subject': ['tiếng anh']},
  'confirm_obj': {'type_edu': ['chất lượng cao', 'đại trà', 'tiên tiến']},
  'text': 'thầy cho em hỏi nếu mình đã trúng tuyển chương trình đại trà thì có thể đổi sang chương trình chất lượng cao tiếng anh hoặc chương trình tiên tiến được không ạ',
  'labeled_intent': 1},
 {'intent': 'register',
  'probabily': 1.0,
  'entity': {'major_name': ['dầu khí', 'dầu khí'], 'point': [2.0, 2.0]},
  'confirm_obj': None,
  'text': 'cho em hỏi em có thể đăng kí 2 ngành được không em đăng kí 2 ngành điện và ngành dầu khí vd trượt ngành điện thì qua dầu khí được không ạ',
  'labeled_intent': 0},
 {'intent': 'type_edu',
  'probabily': 0.9993758797645569,
  'entity': {},
  'confirm_obj': {'type_edu': ['chất lượng cao']},
  'text': 'cho em hỏi chương trình chất lượng cao ở bách khoa có được học hoàn toàn trên cơ sở lý thường kiệt không ạ',
  'labeled_intent': 1},
 {'intent': 'type_edu',
  'probabily': 0.999

In [11]:
file_out = open('/home/taindp/PycharmProjects/thesis/data/result_invest_apr7_v3.json','w')
for item in list_invest:
    if len(item.keys()) > 2:
        item_str = str(item).replace(r"'",r'"')
        file_out.write(item_str)
        file_out.write('\n')